In [2]:
import h5py
import tensorflow as tf
import os
import numpy as np
import json
import shutil

In [3]:
num_episodes = 200
num_files = 20

input_hdf5_dir = '/media/sbr-tech/Desk SSD/aloha/datas_couple_mobile'
input_hdf5_basename = "episode_{}.hdf5"

output_root = "/media/sbr-tech/Desk SSD1/aloha/"
dataset_name = "mobile_aloha_sim_couple_dataset_100"
task_name = "mobile_aloha_sim_couple_dataset"
version = "1.0.0"

output_basename = task_name + "-train.tfrecord-{}-of-{}"

mobile = True

In [4]:
output_dir = os.path.join(output_root, dataset_name, task_name, version)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [5]:
episodes_per_file = num_episodes // num_files

with open("dataset_info_template.json", "r") as f:
    dataset_info = json.load(f)

dataset_info["moduleName"] = task_name + "." + task_name
dataset_info["name"] = task_name
dataset_info["version"] = version
dataset_info["splits"][0]["shardLengths"] = [str(episodes_per_file)] * num_files 

with open(os.path.join(output_dir, "dataset_info.json"), "w") as f:
    json.dump(dataset_info, f, ensure_ascii=False, indent=2, sort_keys=True, separators=(',', ': '))

In [6]:
if not mobile:
    ret = shutil.copy("features_template_static.json", os.path.join(output_dir, "features.json"))
else:
    ret = shutil.copy("features_template_mobile.json", os.path.join(output_dir, "features.json"))
ret

'/media/sbr-tech/Desk SSD1/aloha/mobile_aloha_sim_couple_dataset_100/mobile_aloha_sim_couple_dataset/1.0.0/features.json'

In [7]:
sample_hdf5_file = os.path.join(input_hdf5_dir, input_hdf5_basename.format(0))

In [8]:
NUM_EPISODE = 600 if not mobile else 720

IS_FIRSTS = np.zeros(NUM_EPISODE, dtype=int)
IS_FIRSTS[0] = 1
DISCOUNTS = np.ones(NUM_EPISODE, dtype=float)
IS_LASTS = np.zeros(NUM_EPISODE, dtype=int)
IS_LASTS[-1] = 1
LANGUAGE_INSTRUCTION_CUCUMBER = b"pick up the cucumber and put it in the bucket"
LANGUAGE_INSTRUCTIONS_CUCUMBER = np.array([LANGUAGE_INSTRUCTION_CUCUMBER]*NUM_EPISODE)
LANGUAGE_INSTRUCTION_CUBE = b"pick up the red cube and put it in the bucket"
LANGUAGE_INSTRUCTIONS_CUBE = np.array([LANGUAGE_INSTRUCTION_CUBE]*NUM_EPISODE)
REWARDS = np.zeros(NUM_EPISODE, dtype=float)
REWARDS[-1] = 1
IS_TARMINALS = np.zeros(NUM_EPISODE, dtype=int)
IS_TARMINALS[-1] = 1
METADATA = sample_hdf5_file

In [9]:
def _image_bytes_feature(images):
    values = [tf.image.encode_jpeg(image).numpy() for image in images]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))

def _language_bytes_feature(langs):
    values = [lang.tobytes() for lang in langs]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))
    
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [10]:
def numpy_to_tf_example(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions, mobile=False, cube=False):    
    language_instructions = LANGUAGE_INSTRUCTIONS_CUCUMBER if not cube else LANGUAGE_INSTRUCTIONS_CUBE

    if not mobile:
        feature = {
            "steps/is_first": _int64_feature(IS_FIRSTS),
            "steps/action": _float_feature(actions.flatten()),
            "steps/discount": _float_feature(DISCOUNTS),
            "steps/is_last": _int64_feature(IS_LASTS),
            "steps/language_instruction": _language_bytes_feature(language_instructions),
            "steps/rewards": _float_feature(REWARDS),
            "steps/observation/top": _image_bytes_feature(images_top),
            "steps/observation/angle": _image_bytes_feature(images_angle),
            "steps/observation/left_wrist": _image_bytes_feature(images_left_wrist),
            "steps/observation/right_wrist": _image_bytes_feature(images_right_wrist),
            "steps/is_terminal": _int64_feature(IS_TARMINALS),
            "steps/observation/state": _float_feature(states.flatten()),
            "episode_metadata/file_path": _bytes_feature(METADATA.encode())
        }
    else:
        feature = {
            "steps/is_first": _int64_feature(IS_FIRSTS),
            "steps/action": _float_feature(actions.flatten()),
            "steps/discount": _float_feature(DISCOUNTS),
            "steps/is_last": _int64_feature(IS_LASTS),
            "steps/language_instruction": _language_bytes_feature(language_instructions),
            "steps/rewards": _float_feature(REWARDS),
            "steps/observation/angle": _image_bytes_feature(images_angle),
            "steps/observation/left_wrist": _image_bytes_feature(images_left_wrist),
            "steps/observation/right_wrist": _image_bytes_feature(images_right_wrist),
            "steps/is_terminal": _int64_feature(IS_TARMINALS),
            "steps/observation/state": _float_feature(states.flatten()),
            "episode_metadata/file_path": _bytes_feature(METADATA.encode())
        }        

    features = tf.train.Features(feature=feature)
    example = tf.train.Example(features=features)
    
    return example

In [12]:

episode_idx = 0
for file_idx in range(num_files):
    output_tfrecord_file = os.path.join(output_dir, output_basename.format(str(file_idx).zfill(5), str(num_files).zfill(5)))
    if os.path.exists(output_tfrecord_file):
        episode_idx += episodes_per_file
        continue
    with tf.io.TFRecordWriter(output_tfrecord_file) as writer:
        for _ in range(episodes_per_file):
            hdf5_file = os.path.join(input_hdf5_dir, input_hdf5_basename.format(episode_idx))
            with h5py.File(hdf5_file, 'r') as hdf5_f:
                states = hdf5_f["observations"]["qpos"][:].copy()
                images_top = hdf5_f["observations"]["images"]["top"][:].copy()
                images_angle = hdf5_f["observations"]["images"]["angle"][:].copy()
                images_left_wrist = hdf5_f["observations"]["images"]["left_wrist"][:].copy()
                images_right_wrist = hdf5_f["observations"]["images"]["right_wrist"][:].copy()
                actions = hdf5_f["action"][:].copy()
            
            cube = episode_idx%2 == 1
            example = numpy_to_tf_example(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions, mobile=mobile, cube=cube)

            writer.write(example.SerializeToString())
            episode_idx += 1
